Beschreibung: Colab-Notebook
https://colab.research.google.com/github/Clemes123/exQA-Ausprobieren/Question-Answering-Pipeline.ipynb //funktioniert nur wenn public
(sonst zum Testen einfach Colab starten und dort unter "Upload" dieses Notebook hochladen)

ACHTUNG: In Colab muss unter "Runtime" -> "Change runtime type" die
Hardwarebeschleunigung auf GPU gesetzt werden, sonst dauert die Indizierung
der Dokumente mit Dense Passage Retrieval sehr lange.

## Haystack aufsetzen

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,ocr,preprocessing,file-conversion,pdf]

## Document Store befüllen

### Dokumente konvertieren

In [ ]:
from haystack.utils import convert_files_to_docs
from haystack.nodes import PreProcessor

document_dir = "./LHM_website_dataset"
all_docs = convert_files_to_docs(dir_path=document_dir)

preprocessor = PreProcessor(
    split_by="word",
    split_length=100,
    split_overlap=10,
    split_respect_sentence_boundary=True,
)

docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

### zu Document Store hinzufügen

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()
document_store.write_documents(docs)

### mit Dense Passage Retrieval indizieren
Hier wird für das `query_embedding_model` und `passage_embedding_model` ein
DPR-Modell-Paar eingesetzt, das speziell auf den LHM-Datensatz feinjustiert
wurde. Für allgemeine Anwendungsfälle empfehlen wir folgendes Modell-Paar:
```
query_embedding_model="deepset/gbert-base-germandpr-question_encoder",
passage_embedding_model="deepset/gbert-base-germandpr-ctx_encoder",
```
Dabei ist zu beachten, dass Dense-Passage-Retrieval auf Texten mit
dem Modell unbekannten Fachwörtern nicht gut funktioniert. In diesem Fall
sollte ein eigenes DPR-Modell feinjustiert werden (siehe Artikel siehe
Artikel "Dense Passage Retrieval für die eigene Domäne" aus Heft iX 6/2023).

Falls das nicht möglich ist, bietet sich alternativ der klassische
stichwortbasierte BM25-Algorithmus an. Dafür in der vorangegangenen Zelle
`use_bm25` in `document_store = InMemoryDocumentStore(use_bm25=True)` auf
`True` setzen und die nachfolgende Zelle nicht ausführen.

In [ ]:
from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever(document_store,
                                      query_embedding_model="schreon/xnext-lhm_queries_encoder",
                                      passage_embedding_model="schreon/xnext-lhm_passages_encoder",
                                      embed_title=False)

document_store.update_embeddings(retriever)

## Reader-Komponente mit QA-Modell initialisieren

In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/gelectra-base-germanquad", use_gpu=True)

## Pipeline instanziieren

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipeline = ExtractiveQAPipeline(reader, retriever)

## Pipeline ausführen

In [ ]:
prediction = pipeline.run(
    query="Was darf ich mit einem Jagdschein?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

In [ ]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum"   ## `minimum`, `medium` oder `all`
)